# Capturing Neptune SPARQL queries with Pandas dataframes using the air-routes dataset



## Introduction

In this prototype, I use Pandas dataframes to capture the results of a Neptune SPARQL queries, which opens up the data exploration capabities of Pandas. Pandas deals with tabular data while Neptune is a graph database. Apples and oranges. But where query outputs lend themselvs to tabular data processing, Pandas can come in handy. 

This Notebook was originally generated by the Amazon Neptune Getting Started Guide which provisions a Neptune cluster with an endpoint as well as a number of Jupyter Notebooks with Gremlin and Sparql examples. https://docs.aws.amazon.com/neptune/latest/userguide/get-started.html 

To capture SPARQL query output to Pandas dataframes, I use sparql_dataframe https://pypi.org/project/sparql-dataframe/ 

You can find the original air routs dataset in CSV form at the following GitHub repository: https://github.com/krlawrence/graph.

The RDF version of the data set was created by converting the property graph CSV files into N-Quad format files using the tool located at https://github.com/aws/amazon-neptune-csv-to-rdf-converter


In [1]:
# you need only to execute the below pip command once to install the package. 
#!pip install sparql-dataframe

import pandas as pd
import sparql_dataframe

import json
from SPARQLWrapper import SPARQLWrapper, JSON

def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

endpoint = "https://neptunedbcluster-YOURCLUSTER.cluster-YOURCLUSTER.us-east-1.neptune.amazonaws.com:8182/sparql"


qr = """
PREFIX prop:  <http://kelvinlawrence.net/air-routes/datatypeProperty/>
PREFIX op:    <http://kelvinlawrence.net/air-routes/objectProperty/>
        
SELECT ?from ?d1 ?stop1 ?d2 ?stop2 ?d3 ?to ?total
WHERE {
    ?s  prop:code "AUS" .
    ?s  op:route ?s1 .
    ?s1 op:route ?s2 .
    ?s2 op:route ?s3 .
    ?s3 prop:code "WLG" .
    ?s  prop:code ?from .
    ?s1 prop:code ?stop1 .
    ?s2 prop:code ?stop2 .
    ?s3 prop:code ?to .
    GRAPH ?r1 { ?s op:route ?s1 }
    ?r1 prop:dist ?d1 
    GRAPH ?r2 { ?s1 op:route ?s2 }
    ?r2 prop:dist ?d2 
    GRAPH ?r3 { ?s2 op:route ?s3 }
    ?r3 prop:dist ?d3 
    BIND(?d1 + ?d2 + ?d3 as ?total)
   
}
ORDER BY ?total
LIMIT 20
"""
df = get_sparql_dataframe(endpoint, qr)
df
#df.to_csv('mbx-out.csv')

,from,d1,stop1,d2,stop2,d3,to,total
0,AUS,142,IAH,7416,AKL,298,WLG,7856
1,AUS,1230,LAX,6512,AKL,298,WLG,8040
2,AUS,1500,SFO,6525,AKL,298,WLG,8323
3,AUS,973,ORD,8186,AKL,298,WLG,9457
4,AUS,1230,LAX,7162,BNE,1559,WLG,9951
5,AUS,1230,LAX,7489,SYD,1383,WLG,10102
6,AUS,142,IAH,8591,SYD,1383,WLG,10116
7,AUS,190,DFW,8574,SYD,1383,WLG,10147
8,AUS,1500,SFO,7420,SYD,1383,WLG,10303
9,AUS,1230,LAX,7922,MEL,1608,WLG,10760


The query below produces a nice "star graph" picture showing the routes available from Cozumel and labels the destinations with the airport codes. Try running the query and selecting the Graph tab.

In [2]:

qr = """
PREFIX prop:  <http://kelvinlawrence.net/air-routes/datatypeProperty/>
PREFIX op:    <http://kelvinlawrence.net/air-routes/objectProperty/>

SELECT ?s ?p ?o 
WHERE {
    ?from prop:code "CZM" .
    ?from op:route ?to .
    ?to prop:code ?o .
    ?from prop:code ?s .
    BIND("route" AS ?p)
} 
LIMIT 50
"""
df = get_sparql_dataframe(endpoint, qr)
df


,s,p,o
0,CZM,route,IAH
1,CZM,route,MTY
2,CZM,route,MEX
3,CZM,route,DFW
4,CZM,route,YUL
5,CZM,route,CUN
6,CZM,route,ATL
7,CZM,route,MIA
8,CZM,route,DEN
9,CZM,route,YYZ


The query below returns a table showing the cities where the airports with non-stop flights from Cozumel are located.

In [3]:
qr="""
PREFIX prop: <http://kelvinlawrence.net/air-routes/datatypeProperty/> 
PREFIX op:   <http://kelvinlawrence.net/air-routes/objectProperty/> 

SELECT ?s ?p ?o WHERE {
  ?czm prop:code "CZM" .
  ?czm op:route ?dest .
  ?czm prop:code ?s.
  ?dest prop:code ?o.
  GRAPH ?route { ?czm op:route ?dest }
  ?route prop:dist ?p }
ORDER BY ?p
"""
df = get_sparql_dataframe(endpoint, qr)
df

,s,p,o
0,CZM,36,CUN
1,CZM,556,MIA
2,CZM,792,MEX
3,CZM,838,IAH
4,CZM,911,MTY
5,CZM,918,ATL
6,CZM,1056,DFW
7,CZM,1077,CLT
8,CZM,1653,YYZ
9,CZM,1697,DEN
